<a href="https://colab.research.google.com/github/YazzRz/TFM_Ana_Rozo_Georeferenciacion_Violencia_Machista/blob/main/Validaci%C3%B3n_de_datos_geogr%C3%A1ficos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy geopy
!python -m spacy download es_core_news_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 15.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!python -m spacy download es_core_news_md
!python -m spacy download es_core_news_sm
!pip install newspaper3k
!pip install lxml_html_clean
!pip install geotext


  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_md-3.8.0/es_core_news_md-3.8.0-py3-none-any.whl (42.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from geopy.geocoders import Nominatim
import pandas as pd
from collections import Counter
import time

# Cargar modelo en español
nlp = spacy.load("es_core_news_md")

# Lista de países hispanohablantes (Latam + España)
paises_hispanos = [
    "Argentina", "Bolivia", "Chile", "Colombia", "Costa Rica", "Cuba",
    "Ecuador", "El Salvador", "España", "Guatemala", "Honduras",
    "México", "Nicaragua", "Panamá", "Paraguay", "Perú", "Puerto Rico",
    "República Dominicana", "Uruguay", "Venezuela"
]

# Texto de ejemplo
texto = """La violencia intrafamiliar en Colombia es un problema que se hace más visible a medida que pasan los años. En 2013 la Policía Nacional reportó 33.086 casos en el país, en 2014 registró 48.427 y en 2015 la cifra ascendió a 74.972. Antioquia también presenta un incremento similar: de 1.649 casos en 2013 pasó a 10.569 en 2015.

El aumento en las denuncias es positivo, dice Yolanda Puyana, experta en temas de género y familia: “Las cifras expresan que está siendo mejor la atención a la violencia intrafamiliar. El estudio que realizó Profamilia en 2010 demuestra que apenas el 20 % de las personas denuncia, por eso es importante el incremento en las denuncias”.

La secretaria de la Mujeres en Medellín, Gloria Luz Gómez, dice que el aumento también se debe a que se han capacitado a los funcionarios públicos y se han fortalecido “los dispositivos de atención” . Además, dice, se han ampliado los canales de justicia. Del estudio entregado por la Policía y la Dijín se desprenden las siguientes cifras: en 2015, por violencia intrafamiliar, fueron atacadas 62.565 mujeres, mientras que en 2014 fueron 40.481 . De otro lado, 12.396 denuncias reportados en 2015 corresponden a hombres, 4.450 más de las que se registraron en 2014 (ver infografía).

También preocupan los asesinatos. El año pasado la Policía registró 51 homicidios de mujeres por violencia intrafamiliar y 28 de hombres. Esto significa un aumento con respecto a 2014, año en el que hubo 41 de mujeres y 33 de hombres. Bogotá (13.295 denuncias), Cali (3.269 denuncias) y Medellín (2.912 denuncias) fueron las ciudades con cifras más altas.

La organización Casa de la Mujer sostiene que persiste el subregistro de casos porque existe un “numero significativo de mujeres que no denuncia violencia intrafamiliar, bien sea porque no cuenta con redes de apoyo, porque dependen económicamente de los agresores, tienen hijos a su cargo, están amenazadas o no conocen la institucionalidad que puede asistirlas”.

Además, dice Erika Rodríguez, abogada de la organización, “aún no se cuenta con un sistema de información coordinado entre las instituciones del Estado que dé cuenta de la magnitud real del problema, así, una mujer puede denunciar los hechos ante entidades diversas (Comisarías de familia, casas de justicia, Fiscalía- URI) sin que estas se articulen o coordinen para registrar y atender el caso”.

Reforma a las comisarías
Para que las comisarías de familia puedan atender efectivamente los casos de violencia intrafamiliar, el Ministerio de Justicia, el Instituto Colombiano de Bienestar Familiar (Icbf) y la representante Ángela Robledo se han reunido para elaborar un proyecto de ley que trasforme las comisarías. Este podría ser presentado en el segundo semestre.

Lina Patricia Rodríguez, coordinadora de autoridades administrativas del Icbf, ha estado cerca del proceso: “Desde 2015 trabajamos en una mesa interinstitucional para solucionar algunas complejidades en las comisarías. Ahora estamos trabajando en el tema de competencias. Si se eliminan las conciliaciones de las comisarías éstas podrían atender de manera más efectiva la violencia intrafamiliar. De la conciliación se pueden encargar las procuradurías o los centros de conciliación. Hacen falta por trabajar otros aspectos como la naturaleza de las comisarías, el mejoramiento de infraestructura, el financiamiento y el sistema de información”."""

# Procesar con spaCy
doc = nlp(texto)

# Extraer entidades GPE y LOC
lugares_detectados = []
for ent in doc.ents:
    if ent.label_ in ["GPE", "LOC"]:
        texto_ent = ent.text.strip()
        # Solo palabras únicas o países hispanohablantes
        if texto_ent in paises_hispanos or len(texto_ent.split()) == 1:
            lugares_detectados.append(texto_ent)

# Contar repeticiones y elegir la más frecuente
if not lugares_detectados:
    print("No se detectaron lugares")
else:
    contador = Counter(lugares_detectados)
    lugar_principal = contador.most_common(1)[0][0]

    # Geolocalizador
    geolocator = Nominatim(user_agent="geo_locator", timeout=10)

    jerarquia = {
        "Texto_detectado": "",
        "País": "",
        "Departamento": "",
        "Municipio": "",
        "Dirección completa": ""
    }

    try:
        loc = geolocator.geocode(lugar_principal, language="es", addressdetails=True)
        time.sleep(1)
        if loc:
            direccion = loc.raw.get("address", {})
            pais = direccion.get("country", "")
            # Solo considerar países hispanohablantes
            if pais in paises_hispanos:
                ciudad = direccion.get("city", direccion.get("town", direccion.get("village","")))
                departamento = direccion.get("state", "")

                jerarquia["Texto_detectado"] = ciudad if ciudad else departamento if departamento else pais
                jerarquia["Municipio"] = ciudad
                jerarquia["Departamento"] = departamento
                jerarquia["País"] = pais
                jerarquia["Dirección completa"] = loc.address

    except Exception as e:
        print(f"No se pudo geolocalizar {lugar_principal}: {e}")

    # Pasar a DataFrame
    df_geo = pd.DataFrame([jerarquia])
    print(df_geo)


  Texto_detectado      País Departamento Municipio  \
0        Medellín  Colombia    Antioquia  Medellín   

                                  Dirección completa  
0  Medellín, Valle de Aburrá, Antioquia, RAP del ...  


In [ ]:
import spacy
from geopy.geocoders import Nominatim
import pandas as pd
from collections import Counter
import time
from newspaper import Article

# Cargar modelo en español
nlp = spacy.load("es_core_news_md")

# Países hispanohablantes
paises_hispanos = [
    "Argentina", "Bolivia", "Chile", "Colombia", "Costa Rica", "Cuba",
    "Ecuador", "El Salvador", "España", "Guatemala", "Honduras",
    "México", "Nicaragua", "Panamá", "Paraguay", "Perú", "Puerto Rico",
    "República Dominicana", "Uruguay", "Venezuela"
]

# Leer CSV con URLs
df_urls = pd.read_csv("prueba.csv")

# Preparar geolocalizador
geolocator = Nominatim(user_agent="geo_locator", timeout=10)

# Lista para guardar resultados
resultados = []

for index, row in df_urls.iterrows():
    url = row['url']
    id_noticia = row['id']

    try:
        # Extraer texto de la noticia
        articulo = Article(url, language='es')
        articulo.download()
        articulo.parse()
        texto = articulo.text

        # Procesar con spaCy
        doc = nlp(texto)

        # Extraer entidades GPE y LOC
        lugares_detectados = []
        for ent in doc.ents:
            if ent.label_ in ["GPE", "LOC"]:
                texto_ent = ent.text.strip()
                if texto_ent in paises_hispanos or len(texto_ent.split()) == 1:
                    lugares_detectados.append(texto_ent)

        if lugares_detectados:
            contador = Counter(lugares_detectados)
            lugar_principal = contador.most_common(1)[0][0]

            jerarquia = {
                "id": id_noticia,
                "url": url,
                "Texto_detectado": "",
                "País": "",
                "Departamento": "",
                "Municipio": "",
                "Dirección completa": "",
                "Latitud": "",
                "Longitud": ""
            }

            # Geolocalizar
            try:
                loc = geolocator.geocode(lugar_principal, language="es", addressdetails=True)
                time.sleep(1)
                if loc:
                    direccion = loc.raw.get("address", {})
                    pais = direccion.get("country", "")
                    if pais in paises_hispanos:
                        ciudad = direccion.get("city", direccion.get("town", direccion.get("village","")))
                        departamento = direccion.get("state", "")

                        jerarquia["Texto_detectado"] = ciudad if ciudad else departamento if departamento else pais
                        jerarquia["Municipio"] = ciudad
                        jerarquia["Departamento"] = departamento
                        jerarquia["País"] = pais
                        jerarquia["Dirección completa"] = loc.address

            except Exception as e:
                print(f"No se pudo geolocalizar {lugar_principal}: {e}")

            resultados.append(jerarquia)

    except Exception as e:
        print(f"No se pudo extraer texto de {url}: {e}")

# Convertir a DataFrame solo para visualizar bonito
df_geo = pd.DataFrame(resultados)
print("\n=== RESULTADOS DETECTADOS ===\n")
print(df_geo)


No se pudo extraer texto de https://web.archive.org/web/20200503234339/https://elpais.com/sociedad/2020-03-13/el-caso-renato-ibarra-obliga-al-futbol-mexicano-a-hablar-de-violencia-machista.html: Article `download()` failed with HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20200503234339/https://elpais.com/sociedad/2020-03-13/el-caso-renato-ibarra-obliga-al-futbol-mexicano-a-hablar-de-violencia-machista.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7e970a6c2360>: Failed to establish a new connection: [Errno 111] Connection refused')) on URL https://web.archive.org/web/20200503234339/https://elpais.com/sociedad/2020-03-13/el-caso-renato-ibarra-obliga-al-futbol-mexicano-a-hablar-de-violencia-machista.html
No se pudo extraer texto de https://web.archive.org/web/20200504231009/https://elpais.com/sociedad/2020-03-20/ejecutados-los-cuatro-condenados-por-la-violacion-que-desperto-a-india-contra-la-violencia-mac

In [ ]:

# Guardar resultados en CSV
df_geo.to_csv("resultados_geolocalizados.csv", index=False, encoding="utf-8-sig")
print("\nArchivo 'resultados_geolocalizados.csv' generado correctamente ✅")


Archivo 'resultados_geolocalizados.csv' generado correctamente ✅


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# ======================
# Cargar CSV
# ======================
df = pd.read_csv("ubicaciones_detectadas.csv")

# Normalizar nombres de columnas
df.columns = [c.strip().lower() for c in df.columns]

# ======================
# Configurar geolocalizador con timeout
# ======================
geolocator = Nominatim(user_agent="ubicaciones_app")
geocode = RateLimiter(lambda q: geolocator.geocode(q, language="es", timeout=10), min_delay_seconds=1)

# ======================
# Caché para no repetir consultas
# ======================
cache = {}

def obtener_coordenadas(query):
    # Evitar NaN o valores vacíos
    if pd.isna(query) or str(query).strip() == "":
        return pd.Series([None, None])

    # Revisar cache
    if query in cache:
        return cache[query]

    try:
        location = geocode(str(query))
        if location:
            coords = pd.Series([location.latitude, location.longitude])
            cache[query] = coords
            return coords
    except Exception:
        return pd.Series([None, None])

    return pd.Series([None, None])


# ======================
# 1. Coordenadas del país
# ======================
df[["lat_pais", "lon_pais"]] = df["pais_detectado"].apply(
    lambda x: obtener_coordenadas(x)
)

# ======================
# 2. Coordenadas del departamento
# ======================
df[["lat_departamento", "lon_departamento"]] = df.apply(
    lambda x: obtener_coordenadas(f"{x['departamento_detectado']}, {x['pais_detectado']}")
    if pd.notna(x["departamento_detectado"]) and x["departamento_detectado"].strip() != ""
    else pd.Series([None, None]),
    axis=1
)

# ======================
# 3. Coordenadas del municipio
# ======================
df[["lat_municipio", "lon_municipio"]] = df.apply(
    lambda x: obtener_coordenadas(f"{x['municipio_detectado']}, {x['departamento_detectado']}, {x['pais_detectado']}")
    if pd.notna(x["municipio_detectado"]) and x["municipio_detectado"].strip() != ""
    else pd.Series([None, None]),
    axis=1
)

# ======================
# Guardar tabla maestra
# ======================
df.to_csv("tabla_maestra_ubicaciones.csv", index=False, encoding="utf-8")

print("✅ Tabla maestra creada: tabla_maestra_ubicaciones.csv")


✅ Tabla maestra creada: tabla_maestra_ubicaciones.csv
